In [ ]:
#Import necessary libraries
import msal
import requests
import json
from datetime import datetime
from pyspark.sql.functions import col, explode_outer

#### Run another Notebook to get Secret from Azure Key Vault

In [ ]:
%run Get Fabric Access Token

#### Define parameters. If an incremental refresh should happen, modify the URL

In [ ]:
#Set Tenant API URL
url = 'https://api.fabric.microsoft.com/v1/admin/tenantsettings'

#Get Current datetime
now = datetime.now()

#### Define functions for reusabilty

In [ ]:
#Define Request Functions

#Get Request
def get_request(url, header):
    api_call = requests.get(url=url, headers=header)
    return api_call

#Post Reuqest
def post_request(url, header, body):
    api_call = requests.post(url=url, headers=header, json=body)
    return api_call

#### Start calling the API

In [ ]:
#Call API
api_call = get_request(url, header)

#Convert result to JSON and store in a dataframe
result_tenant_settings = api_call.json()['tenantSettings']
df_tenant_settings = spark.createDataFrame(result_tenant_settings)

display(df_tenant_settings)

#### Store Security Group UID and Name into separate columns, creating one row for each group

In [ ]:
#Specify Columns
columns = ["settingName", "title", "enabled", "tenantSettingGroup", "properties", "canSpecifySecurityGroups",]

df_tenant_settings = df_tenant_settings.select(
    *columns,
    explode_outer("enabledSecurityGroups").alias("Exploded_SG")
    ).select(
        *columns,
        col("Exploded_SG.graphId").alias("SGUID"),
        col("Exploded_SG.name").alias("SGName")
        )

# Show the result
display(df_tenant_settings)
